<a href="https://colab.research.google.com/github/ambilynanjilath/deep-learning-projects/blob/master/json_creating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!cp '/content/json_creating.ipynb'   '/content/drive/MyDrive/Data_Scraping'

In [3]:
!git config --global user.email "ambilybiju2408@gmail.com"
!git config --global user.name "ambilynanjilath"


In [4]:
%cd /content/drive/MyDrive/Data_Scraping

/content/drive/MyDrive/Data_Scraping


In [5]:
!git init


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/Data_Scraping/.git/


In [6]:
!git add json_creating.ipynb


In [7]:
!git commit -m "initial commit"

[master (root-commit) a88ca8f] initial commit
 1 file changed, 677 insertions(+)
 create mode 100644 json_creating.ipynb


In [17]:
!ssh-keygen -t rsa -b 4096 -C "ambilybiju2408@gmail.com"

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
Created directory '/root/.ssh'.
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:ZXtne3OnPX4AIYmLGB8sTclm5U2Yz2OhFlscLklZiIU ambilybiju2408@gmail.com
The key's randomart image is:
+---[RSA 4096]----+
|     =.*+O+o     |
|    o EoB=* .    |
|     B o+O=o .   |
|    . o =+=..    |
|       .S.....o  |
|           . o.. |
|              .o+|
|               ==|
|              o.+|
+----[SHA256]-----+


In [19]:
!eval "$(ssh-agent -s)"


Agent pid 13264


In [20]:
!ssh-add ~/.ssh/id_rsa


Could not open a connection to your authentication agent.


In [21]:
!ssh-agent -k && ssh-agent -s

SSH_AGENT_PID not set, cannot kill agent


In [ ]:
#bestest best

In [ ]:
import pandas as pd
df=pd.read_csv('/content/YC_Companies(name,loc,tag).csv')


In [ ]:
urls=df['link'].to_list()

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def scrape_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Scrape company name
        company_name = soup.find('h1', class_='font-extralight').text.strip()

        # Check condition 1
        if soup.find('div', class_='space-y-5'):
            founders = []
            founders_divs = soup.find_all('div', class_='flex flex-row flex-col items-start gap-3 md:flex-row')
            for founder_div in founders_divs:
                founder_info = {'name': '', 'role': '', 'description': '', 'linkedin_profile': '', 'twitter_profile': ''}
                name_role_desc_div = founder_div.find('div', class_='flex-grow')
                if name_role_desc_div:
                    name_role = name_role_desc_div.find('h3', class_='text-lg font-bold')
                    description = name_role_desc_div.find('p', class_='prose max-w-full whitespace-pre-line')
                    if name_role:
                        name_role_parts = name_role.text.split(',', 1)
                        if len(name_role_parts) >= 2:
                            founder_info['name'] = name_role_parts[0].strip()
                            founder_info['role'] = name_role_parts[1].strip()
                        elif len(name_role_parts) == 1:
                            founder_info['name'] = name_role_parts[0].strip()
                    if description:
                        founder_info['description'] = description.text.strip()

                linkedin_twitter_div = founder_div.find('div', class_='shrink-0 space-y-1.5 rounded-md border-[1px] border-[#999] bg-[#FDFDF7] p-6 sm:w-[300px]')
                if linkedin_twitter_div:
                    social_links = linkedin_twitter_div.find('div', class_='mt-1 space-x-2')
                    linkedin_tag = social_links.find('a', class_='bg-image-linkedin')
                    twitter_tag = social_links.find('a', class_='bg-image-twitter')
                    if linkedin_tag:
                        founder_info['linkedin_profile'] = linkedin_tag['href']
                    if twitter_tag:
                        founder_info['twitter_profile'] = twitter_tag['href']

                founders.append(founder_info)

            return {'company': {'name': company_name}, 'founders': founders}

        # Check condition 2
        elif soup.find('div', class_='space-y-4'):
            founders = []
            founder_divs = soup.find_all('div', class_='shrink-0 space-y-1.5 rounded-md border-[1px] border-[#999] bg-[#FDFDF7] p-6 sm:w-[300px]')
            for founder_div in founder_divs:
                founder_info = {'name': '', 'role': '', 'description': '', 'linkedin_profile': '', 'twitter_profile': ''}
                name = founder_div.find('div', class_='font-bold').text.strip()
                role = founder_div.find('div').text.strip()
                linkedin_tag = founder_div.find('a', class_='bg-image-linkedin')
                twitter_tag = founder_div.find('a', class_='bg-image-twitter')
                founder_info['name'] = name
                founder_info['role'] = role
                if linkedin_tag:
                    founder_info['linkedin_profile'] = linkedin_tag['href']
                if twitter_tag:
                    founder_info['twitter_profile'] = twitter_tag['href']
                founders.append(founder_info)

            return {'company': {'name': company_name}, 'founders': founders}

        else:
            # If neither condition 1 nor condition 2 is satisfied
            return {'company': {'name': company_name}, 'founders': []}
    else:
        print("Failed to fetch page:", url)
        return None


data = []
for url in urls:
    result = scrape_data(url)
    if result:
        data.append(result)

# Write data to JSON file
with open('founder_data.json', 'w') as f:
    json.dump(data, f, indent=4)

print("Data scraped and saved to 'scraped_data.json'")


Data scraped and saved to 'scraped_data.json'


In [ ]:
import json

# Load data from Company_data.json
with open('Company_data.json', 'r') as company_file:
    company_data = json.load(company_file)

# Load data from founder_data.json
with open('founder_data.json', 'r') as founder_file:
    founder_data = json.load(founder_file)

# Initialize a dictionary to store combined data
combined_data = {}

# Extract company information
for company_entry in company_data['companies']:
    company_name = company_entry['name']
    combined_data['company'] = {
        "name": company_name,
        "tagline": company_entry['tagline'],
        "description": company_entry['description'],
        "batch": company_entry['batch'],
        "company_type": company_entry['company_type'],
        "industry_tags": company_entry['industry_tags'],
        "location": company_entry['location'],
        "website": company_entry['website'],
        "founded": company_entry['founded'],
        "team_size": company_entry['team_size'],
        "social_profiles": company_entry['social_profiles']
    }

# Extract founder information
for founder_entry in founder_data:
    if founder_entry['company']['name'] == combined_data['company']['name']:
        combined_data['founders'] = []
        for founder in founder_entry['founders']:
            combined_data['founders'].append({
                "name": founder['name'],
                "description": founder['description'],
                "twitter_profile": founder['twitter_profile'],
                "linkedin_profile": founder['linkedin_profile']
            })

# Write combined data to a new JSON file
with open('yc_companies.json', 'w') as combined_file:
    json.dump(combined_data, combined_file, indent=4)


In [ ]:
import json

# Load data from Company_data.json
with open('Company_data.json', 'r') as company_file:
    company_data = json.load(company_file)

# Load data from founder_data.json
with open('founder_data.json', 'r') as founder_file:
    founder_data = json.load(founder_file)

# Initialize a list to store combined data for multiple companies
combined_data_list = []

# Extract company and founder information
for company_entry in company_data['companies']:
    company_name = company_entry['name']
    combined_company_data = {
        "company": {
            "name": company_name,
            "tagline": company_entry['tagline'],
            "description": company_entry['description'],
            "batch": company_entry['batch'],
            "company_type": company_entry['company_type'],
            "industry_tags": company_entry['industry_tags'],
            "location": company_entry['location'],
            "website": company_entry['website'],
            "founded": company_entry['founded'],
            "team_size": company_entry['team_size'],
            "social_profiles": company_entry['social_profiles']
        },
        "founders": []
    }

    for founder_entry in founder_data:
        if founder_entry['company']['name'] == company_name:
            for founder in founder_entry['founders']:
                combined_company_data['founders'].append({
                    "name": founder['name'],
                    "description": founder['description'],
                    "twitter_profile": founder['twitter_profile'],
                    "linkedin_profile": founder['linkedin_profile']
                })

    combined_data_list.append(combined_company_data)

# Write combined data to a new JSON file
with open('YC_Company.json', 'w') as combined_file:
    json.dump(combined_data_list, combined_file, indent=4)


In [1]:
import json

# Load the JSON file
with open('/content/YC_Company.json', 'r') as f:
    data = json.load(f)

# Count the elements
num_elements = len(data)

print("Number of elements in the JSON file:", num_elements)


Number of elements in the JSON file: 4666


In [ ]:
#best1

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def scrape_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Scrape company name
        company_name = soup.find('h1', class_='font-extralight').text.strip()

        # Check condition 1
        if soup.find('div', class_='space-y-5'):
            founders = []
            founders_divs = soup.find_all('div', class_='flex flex-row flex-col items-start gap-3 md:flex-row')
            for founder_div in founders_divs:
                founder_info = {}
                name_role_desc_div = founder_div.find('div', class_='flex-grow')
                if name_role_desc_div:
                    name_role = name_role_desc_div.find('h3', class_='text-lg font-bold')
                    description = name_role_desc_div.find('p', class_='prose max-w-full whitespace-pre-line')
                    if name_role:
                        name_role_parts = name_role.text.split(',', 1)
                        if len(name_role_parts) >= 2:
                            founder_info['name'] = name_role_parts[0].strip()
                            founder_info['role'] = name_role_parts[1].strip()
                        elif len(name_role_parts) == 1:
                            founder_info['name'] = name_role_parts[0].strip()
                            founder_info['role'] = ''
                    if description:
                        founder_info['description'] = description.text.strip()

                linkedin_twitter_div = founder_div.find('div', class_='shrink-0 space-y-1.5 rounded-md border-[1px] border-[#999] bg-[#FDFDF7] p-6 sm:w-[300px]')
                if linkedin_twitter_div:
                    social_links = linkedin_twitter_div.find('div', class_='mt-1 space-x-2')
                    linkedin_tag = social_links.find('a', class_='bg-image-linkedin')
                    twitter_tag = social_links.find('a', class_='bg-image-twitter')
                    if linkedin_tag:
                        founder_info['linkedin_profile'] = linkedin_tag['href']
                    if twitter_tag:
                        founder_info['twitter_profile'] = twitter_tag['href']

                founders.append(founder_info)

            return {'company': {'name': company_name}, 'founders': founders}

        # Check condition 2
        elif soup.find('div', class_='space-y-4'):
            founders = []
            founder_divs = soup.find_all('div', class_='shrink-0 space-y-1.5 rounded-md border-[1px] border-[#999] bg-[#FDFDF7] p-6 sm:w-[300px]')
            for founder_div in founder_divs:
                founder_info = {}
                name = founder_div.find('div', class_='font-bold').text.strip()
                role = founder_div.find('div').text.strip()
                linkedin_tag = founder_div.find('a', class_='bg-image-linkedin')
                twitter_tag = founder_div.find('a', class_='bg-image-twitter')
                founder_info['name'] = name
                founder_info['role'] = role
                if linkedin_tag:
                    founder_info['linkedin_profile'] = linkedin_tag['href']
                if twitter_tag:
                    founder_info['twitter_profile'] = twitter_tag['href']
                founders.append(founder_info)

            return {'company': {'name': company_name}, 'founders': founders}

        else:
            # If neither condition 1 nor condition 2 is satisfied
            return {'company': {'name': company_name}, 'founders': []}
    else:
        print("Failed to fetch page:", url)
        return None

urls = [
    "https://www.ycombinator.com/companies/homeflow",
    "https://www.ycombinator.com/companies/finta",
    "https://www.ycombinator.com/companies/intercept",
    "https://www.ycombinator.com/companies/fern",
    "https://www.ycombinator.com/companies/axle",
    "https://www.ycombinator.com/companies/hona",
    "https://www.ycombinator.com/companies/quivr",
    "https://www.ycombinator.com/companies/capsule",
    "https://www.ycombinator.com/companies/stralis-aircraft",
    "https://www.ycombinator.com/companies/flex",
    "https://www.ycombinator.com/companies/corefin",
    "https://www.ycombinator.com/companies/affinity",
    "https://www.ycombinator.com/companies/thera",
    "https://www.ycombinator.com/companies/craze",
    "https://www.ycombinator.com/companies/decohere"
]


data = []
for url in urls:
    result = scrape_data(url)
    if result:
        data.append(result)

# Write data to JSON file
with open('scraped_data.json', 'w') as f:
    json.dump(data, f, indent=4)

print("Data scraped and saved to 'scraped_data.json'")


Data scraped and saved to 'scraped_data.json'


In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def scrape_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Scrape company name
        company_name = soup.find('h1', class_='font-extralight').text.strip()

        # Check condition 1
        if soup.find('div', class_='space-y-5'):
            founders = []
            founders_divs = soup.find_all('div', class_='flex flex-row flex-col items-start gap-3 md:flex-row')
            for founder_div in founders_divs:
                founder_info = {}
                name_role_desc_div = founder_div.find('div', class_='flex-grow')
                if name_role_desc_div:
                    name_role = name_role_desc_div.find('h3', class_='text-lg font-bold')
                    description = name_role_desc_div.find('p', class_='prose max-w-full whitespace-pre-line')
                    if name_role:
                        name_role_parts = name_role.text.split(',', 1)
                        if len(name_role_parts) >= 2:
                            founder_info['name'] = name_role_parts[0].strip()
                            founder_info['role'] = name_role_parts[1].strip()  # Extract role here
                        elif len(name_role_parts) == 1:
                            founder_info['name'] = name_role_parts[0].strip()
                            founder_info['role'] = ''
                    if description:
                        founder_info['description'] = description.text.strip()

                linkedin_twitter_div = founder_div.find('div', class_='shrink-0 space-y-1.5 rounded-md border-[1px] border-[#999] bg-[#FDFDF7] p-6 sm:w-[300px]')
                if linkedin_twitter_div:
                    social_links = linkedin_twitter_div.find('div', class_='mt-1 space-x-2')
                    linkedin_tag = social_links.find('a', class_='bg-image-linkedin')
                    twitter_tag = social_links.find('a', class_='bg-image-twitter')
                    if linkedin_tag:
                        founder_info['linkedin_profile'] = linkedin_tag['href']
                    if twitter_tag:
                        founder_info['twitter_profile'] = twitter_tag['href']

                founders.append(founder_info)

            return {'company': {'name': company_name}, 'founders': founders}

        # Check condition 2
        elif soup.find('div', class_='space-y-4'):
            founders = []
            founder_divs = soup.find_all('div', class_='shrink-0 space-y-1.5 rounded-md border-[1px] border-[#999] bg-[#FDFDF7] p-6 sm:w-[300px]')
            for founder_div in founder_divs:
                founder_info = {}
                name_role_div = founder_div.find('h3', class_='text-lg font-bold')
                if name_role_div:
                    name_role_parts = name_role_div.text.split(',', 1)
                    if len(name_role_parts) >= 2:
                        founder_info['name'] = name_role_parts[0].strip()
                        founder_info['role'] = name_role_parts[1].strip()  # Extract role here
                    else:
                        founder_info['name'] = name_role_div.text.strip()
                        founder_info['role'] = ''  # No role found, set to empty string
                linkedin_tag = founder_div.find('a', class_='bg-image-linkedin')
                twitter_tag = founder_div.find('a', class_='bg-image-twitter')
                if linkedin_tag:
                    founder_info['linkedin_profile'] = linkedin_tag['href']
                if twitter_tag:
                    founder_info['twitter_profile'] = twitter_tag['href']
                founders.append(founder_info)

            return {'company': {'name': company_name}, 'founders': founders}

        else:
            # If neither condition 1 nor condition 2 is satisfied
            return {'company': {'name': company_name}, 'founders': []}
    else:
        print("Failed to fetch page:", url)
        return None

urls = [
    "https://www.ycombinator.com/companies/homeflow",
    "https://www.ycombinator.com/companies/finta",
    "https://www.ycombinator.com/companies/intercept",
    "https://www.ycombinator.com/companies/fern",
    "https://www.ycombinator.com/companies/axle",
    "https://www.ycombinator.com/companies/hona",
    "https://www.ycombinator.com/companies/quivr",
    "https://www.ycombinator.com/companies/capsule",
    "https://www.ycombinator.com/companies/stralis-aircraft",
    "https://www.ycombinator.com/companies/flex",
    "https://www.ycombinator.com/companies/corefin",
    "https://www.ycombinator.com/companies/affinity",
    "https://www.ycombinator.com/companies/thera",
    "https://www.ycombinator.com/companies/craze",
    "https://www.ycombinator.com/companies/decohere"
]

data = []
for url in urls:
    result = scrape_data(url)
    if result:
        data.append(result)

# Write data to JSON file
with open('scraped_data.json', 'w') as f:
    json.dump(data, f, indent=4)

print("Data scraped and saved to 'scraped_data.json'")


Data scraped and saved to 'scraped_data.json'


In [ ]:
#################OPENING THE WEBSITE####################################################

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

driver.maximize_window()
driver.get("https://www.ycombinator.com/companies")

# Wait for 2 minutes (120 seconds)
time.sleep(120)

# Close the browser window
driver.quit()




In [ ]:


##############################OPENING WEBSITE INSIDE WEBSITE######################################

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

driver.maximize_window()
driver.get("https://www.ycombinator.com/companies")

# Wait for the page to load
time.sleep(30)  # You can adjust this time if needed

# Define the class name of the element to click
element_class_name ="_coName_99gj3_454"

# Wait for the element to be clickable
wait = WebDriverWait(driver, 60)  # Maximum wait time of 60 seconds
element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, element_class_name)))

# Click on the element
element.click()

# Wait for 2 minutes (120 seconds)
time.sleep(120)

# Close the browser window
driver.quit()





In [ ]:

##################### CLICKING AND GO BACK###############################################



from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

driver.maximize_window()
driver.get("https://www.ycombinator.com/companies")

# Wait for the page to load (adjust if needed)
time.sleep(30)

# Define the class name of the element to click
element_class_name = "_coName_99gj3_454"  # Update with actual class name

# Wait for the element to be clickable
wait = WebDriverWait(driver, 60)
element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, element_class_name)))

# Click on the element
element.click()

# Go back to the previous page
driver.back()

# Wait for 15 seconds after going back
time.sleep(15)

# Close the browser window
driver.quit()



In [ ]:

####################### clicking multiple company ############################################

# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from webdriver_manager.chrome import ChromeDriverManager
# import time

# # Increase wait times as needed for slower internet or complex pages
# wait_time_for_load = 10
# wait_time_per_company = 2

# chrome_options = Options()
# chrome_options.add_experimental_option("detach", True)
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# driver.maximize_window()
# driver.get("https://www.ycombinator.com/companies")

# # Wait for the page to load with expected conditions (preferred over time.sleep)
# wait = WebDriverWait(driver, wait_time_for_load)
# wait.until(EC.presence_of_element_located((By.CLASS_NAME, "_coName_99gj3_454")))  # Update if needed

# # Find elements with the class name (replace with the actual class)
# company_elements = driver.find_elements(By.CLASS_NAME, "_coName_99gj3_454")

# # Keep track of visited websites
# visited_count = 0

# for element in company_elements:
#   # Click on the element
#   element.click()
#   visited_count += 1  # Increment counter on each visit

#   # Wait for the company page to load (adjust if needed)
#   time.sleep(wait_time_for_load)

#   # Go back to the previous page (Y Combinator companies list)
#   driver.back()

#   # Wait after going back
#   time.sleep(wait_time_per_company)

# # Print the total count after processing
# print(f"Total websites visited: {visited_count}")

# # Close the browser window
# driver.quit()





###################### NAME SCRAPING ######################

# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from webdriver_manager.chrome import ChromeDriverManager
# import time

# # Increase wait times as needed for slower internet or complex pages
# wait_time_for_load = 10
# wait_time_per_company = 2

# chrome_options = Options()
# chrome_options.add_experimental_option("detach", True)
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# driver.maximize_window()
# driver.get("https://www.ycombinator.com/companies")

# # Wait for the page to load with expected conditions
# wait = WebDriverWait(driver, wait_time_for_load)
# wait.until(EC.presence_of_element_located((By.CLASS_NAME, "_coName_99gj3_454")))  # Update if needed

# # Find company elements
# company_elements = driver.find_elements(By.CLASS_NAME, "_coName_99gj3_454")

# # Keep track of visited websites and scraped names
# visited_count = 0
# scraped_names = []

# for element in company_elements:
#   # Click on the element
#   element.click()
#   visited_count += 1

#   # Wait for the company page to load (adjust if needed)
#   time.sleep(wait_time_for_load)

#   try:
#     # Find the element with the company name (update class name if needed)
#     company_name_element = driver.find_element(By.CLASS_NAME, "font-extralight")
#     # Get the text content of the element
#     company_name = company_name_element.text.strip()
#     scraped_names.append(company_name)
#     print(f"Scraped name: {company_name}")
#   except:
#     print(f"Couldn't find company name element for company {visited_count}")

#   # Go back to the previous page
#   driver.back()

#   # Wait after going back
#   time.sleep(wait_time_per_company)

# # Print the total count after processing
# print(f"Total companies visited: {visited_count}")
# print(f"Total names scraped: {len(scraped_names)}")

# # Close the browser window
# driver.quit()




